In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''
grid_size = 200


In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'H0TKAGET3ZH984MA',
  'HostId': 'jZrK3YSkjgx3GsyAb6X/p5EO4Avk407kTfL6F0LvL1et4WE0+flqOLYAWkJ1VT/n3iaZA+BFzS4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'jZrK3YSkjgx3GsyAb6X/p5EO4Avk407kTfL6F0LvL1et4WE0+flqOLYAWkJ1VT/n3iaZA+BFzS4=',
   'x-amz-request-id': 'H0TKAGET3ZH984MA',
   'date': 'Tue, 06 May 2025 01:02:24 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': '540362055257-storage-class-analysis',
   'CreationDate': datetime.datetime(2025, 4, 30, 16, 32, 6, tzinfo=tzutc())},
  {'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {

In [5]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=4,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-05-05 20:02:26,108][INFO    ][coiled] Fetching latest package priorities...
[2025-05-05 20:02:26,109][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-05-05 20:02:27,020][INFO    ][coiled.package_sync] Scanning 444 conda packages...
[2025-05-05 20:02:27,029][INFO    ][coiled.package_sync] Scanning 260 python packages...
[2025-05-05 20:02:27,921][INFO    ][coiled] Running pip check...
[2025-05-05 20:02:29,424][INFO    ][coiled] Validating environment...
[2025-05-05 20:02:52,095][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-05-05 20:02:52,313][INFO    ][coiled] Creating wheel for /opt/spark-2.2.0/python...
[2025-05-05 20:02:52,701][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-05-05 20:02:52,703][INFO    ][coiled] Uploading coiled

Started a new Dask client on Coiled. Dashboard is available at https://cluster-qyvxx.dask.host/y22h7KkaTICJJExd/status


In [6]:
import dask_geopandas as dgpd
import dask.dataframe as dd
import pandas as pd
from dask import delayed, compute, visualize
import geopandas as gpd
from dask.diagnostics import ProgressBar
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
#from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection
from scipy.optimize import fminbound, minimize
from metrics_groupby import metrics

from pre_processing import *
from auxiliary_functions import *
from standardize_metrics import *

YOUR_NAME = 'sara'
grid_size = 200


In [7]:
from pre_processing import *

import time
start_time = time.time()  # Start the timer

%autoreload
#from main import *


def preprocess_all_cities(city_list):
    #delayed_jobs = [delayed(calculate_building_distances_to_roads)(city) for city in city_list]
    #delayed_jobs.append([delayed(produce_blocks)(city) for city in city_list])
    delayed_jobs = []
    delayed_jobs.append([calculate_building_distances_to_roads(city) for city in city_list])
    delayed_jobs.append([produce_blocks(city,YOUR_NAME,grid_size) for city in city_list])
    results = compute(*delayed_jobs)
    return results

preprocess_all_cities(["Abidjan", "Accra", "Bamako", "Belo_Horizonte", 
         "Bogota", "Campinas", "Cape_Town", "Lagos",
        "Luanda", "Nairobi","Medellin"]) 
#["Abidjan", "Accra", "Nairobi", "Bamako", "Belo_Horizonte", "Bogota", "Campinas", "Cape_Town", "Lagos","Luanda", "Nairobi", "Medellin"])

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

KeyboardInterrupt: 

In [9]:

from pre_processing import *

import time
start_time = time.time()  # Start the timer


city_list = ["Abidjan", "Accra", "Bamako", "Belo_Horizonte", "Bogota", "Campinas", "Cape_Town", "Lagos","Luanda", "Nairobi","Medellin"]
#["Abidjan", "Accra", "Nairobi", "Bamako", "Belo_Horizonte", "Bogota", "Campinas", "Cape_Town", "Lagos","Luanda", "Nairobi", "Medellin"])

delayed_jobs = []
delayed_jobs.append([produce_azimuths(city, YOUR_NAME, grid_size) for city in city_list])
results = compute(*delayed_jobs)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

Tasks completed in 713.35 seconds.


In [ ]:
from dask import delayed
import dask_geopandas as dgpd
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkb
from scipy.stats import entropy


def building_and_intersection_metrics(city_name):
    grid_cell_count = 0
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
        'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
        'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
        'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
     }
    # Get EPSG
    epsg = get_epsg(city_name).compute()
    # slim to just geometry & persist
    roads = load_dataset(paths['roads'], epsg=epsg)
    grid = load_dataset(paths['grid'], epsg=epsg)#.compute()

    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])
    grid['cell_area'] = grid.geometry.area

    cells = grid.index.size
    grid_cell_count += cells

    # Load buildings and perform relevant calculations on it
    buildings = load_dataset(paths['buildings'], epsg=epsg)#.compute()
    buildings['area'] = buildings.geometry.area
    joined_buildings = dgpd.sjoin(buildings, grid, predicate='intersects')  
    counts_buildings = joined_buildings.groupby('index_right').size()
    
    grid['n_buildings'] = counts_buildings.astype(int)
    grid['n_buildings'] = grid['n_buildings'].fillna(0).astype(int)
    built_area_buildings = joined_buildings.groupby('index_right')['area'].sum()
    grid['built_area'] = built_area_buildings.astype(float)
    grid['built_area'] = grid['built_area'].fillna(0.0)
    
    included_road_types = ['trunk','motorway','primary','secondary','tertiary','primary_link','secondary_link','tertiary_link','trunk_link','motorway_link','residential','unclassified','road','living_street']

    roads_geo = roads[['geometry']]#.persist()

    # tiny grid dataframe for overlay
    grid_small = (
        grid.reset_index()[['index','geometry']]
            .rename(columns={'index':'index_right'})
    )

    # per-partition overlay + length
    def road_length_partition(df, grid_sm):
        clipped = gpd.overlay(df, grid_sm, how='intersection')
        L = clipped.geometry.length.values
        return pd.DataFrame({
            'index_right': clipped['index_right'].values,
            'length_in_cell': L
        }, index=clipped.index)

    meta_rl = pd.DataFrame({
        'index_right': pd.Series(dtype='int64'),
        'length_in_cell': pd.Series(dtype='float64')
    })

    road_parts = roads_geo.map_partitions(
        road_length_partition, grid_small, meta=meta_rl
    ).persist()

    agg_rl = road_parts.groupby('index_right').agg(
        total_len_m=('length_in_cell','sum')
    )

    grid['cell_area_km2'] = grid['cell_area']/1000000.
    
    grid['road_length'] = (agg_rl['total_len_m'] / 1000.)
    grid['road_length'] = grid['road_length'].fillna(0.0)
    grid['has_roads'] = grid['road_length'] > 0

    # Intersection metrics
    intersections = load_dataset(paths['intersections'], epsg=epsg)
    ji = dgpd.sjoin(intersections, grid, predicate='intersects')
    counts2 = ji[ji.street_count>=2].groupby('index_right').size()
    counts3 = ji[ji.street_count>=3].groupby('index_right').size()
    counts4 = ji[ji.street_count==4].groupby('index_right').size()
    
    grid['n_intersections'] = counts2
    grid['n_intersections'] = grid['n_intersections'].fillna(0.).astype(int)

    
    # right after you build n_intersections:
    grid['has_intersections'] = (grid['n_intersections'] > 0).astype('bool')


    
    grid['intersections_3plus'] = counts3
    grid['intersections_3plus'] = grid['intersections_3plus'].fillna(0).astype(int)
    
    grid['intersections_4way']  = counts4
    grid['intersections_4way']  = grid['intersections_4way'].fillna(0).astype(int)

    # Downstream metrics
    grid['m3_raw'] = grid['road_length'] / grid['cell_area_km2']
    grid['m3_std'] = grid['m3_raw'].map_partitions(standardize_metric_3, meta=('m3','float64'))
    grid['m4_raw'] = grid['intersections_4way'] / grid['intersections_3plus']
    grid['m4_raw'] = grid['m4_raw'].mask(
        grid['m4_raw'].isna() & grid['has_roads'], 
        0.0)
    grid['m4_std'] = grid['m4_raw'].map_partitions(standardize_metric_4, meta=('m4','float64'))

    grid['m5_raw'] = (1000**2) * (grid['n_intersections'] / grid['cell_area'])
    grid['m5_raw'] = grid['m5_raw'].mask(
        grid['has_roads'] & grid['m5_raw'].isna(), 
        0.0)
    grid['m5_std'] = grid['m5_raw'].map_partitions(standardize_metric_5, meta=('m5','float64'))
    grid['m10_raw'] = grid['n_buildings'] / grid['cell_area_km2']
    grid['m10_std'] = grid['m10_raw'].map_partitions(standardize_metric_10, meta=('m10','float64'))
    grid['m11_raw'] = grid['built_area'] / grid['cell_area']
    grid['m11_std'] = grid['m11_raw'].map_partitions(standardize_metric_11, meta=('m11','float64'))
    grid['m12_raw'] = grid['built_area'] / grid['n_buildings']
    grid['m12_std'] = grid['m12_raw'].map_partitions(standardize_metric_12, meta=('m12','float64'))

    # Write & return
    out = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_metrics_3_4_5_10_11_12_grid_{YOUR_NAME}.geoparquet'
    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])
    grid.to_parquet(out)
    return out

@delayed
def building_distance_metrics(city_name):
     paths = {
         'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
         'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
         'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
         'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
         'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
     }
     # Get EPSG
     epsg = get_epsg(city_name).compute()
     # Load grid
     grid = load_dataset(paths['grid'], epsg=epsg)#.compute()
     if 'geom' in grid.columns:
         grid = grid.drop(columns=['geom'])
     
     buildings = load_dataset(paths['buildings_with_distances'], epsg=epsg)#.compute()
     buildings['distance_to_nearest_road'] = buildings['distance_to_nearest_road'].astype(float)
     buildings['area'] = buildings.geometry.area
     joined_buildings = dgpd.sjoin(buildings, grid, predicate='intersects')  
     counts_buildings = joined_buildings.groupby('index_right').size()
     grid['n_buildings'] = counts_buildings
     grid['n_buildings'] = grid['n_buildings'].fillna(0).astype(int)

     grid['has_buildings'] = grid['n_buildings']    > 0
     average_distance = joined_buildings.groupby('index_right')['distance_to_nearest_road'].mean()
     grid['average_distance_nearest_building'] = average_distance
     grid['average_distance_nearest_building'] = grid['average_distance_nearest_building'].fillna(0.0)

    
    
     buildings_closer_than_20m = buildings[buildings['distance_to_nearest_road'] <= 20]
     joined_buildings_closer_than_20m = dgpd.sjoin(buildings_closer_than_20m, grid, predicate='intersects') 
     n_buildings_closer_than_20m = joined_buildings_closer_than_20m.groupby('index_right').size()
     grid['n_buildings_closer_than_20m'] = n_buildings_closer_than_20m
     grid['n_buildings_closer_than_20m'] = grid['n_buildings_closer_than_20m'].fillna(0.0)
     grid = grid.assign(
    n_buildings_closer_than_20m = grid['n_buildings_closer_than_20m'].mask(
        (grid['n_buildings'] > 0) & (grid['n_buildings_closer_than_20m'].isna()),
        0))
     grid = grid.assign(
         m1_raw = grid['n_buildings_closer_than_20m'] / grid['n_buildings']
         )
     grid['m1_raw'] = grid['m1_raw']#.fillna(grid['m1_raw'].mean())#grid['n_buildings_closer_than_20m'] / grid['n_buildings']
     grid['m1_std'] = grid['m1_raw'].map_partitions(standardize_metric_1, meta=('m1', 'float64'))
     grid['m2_raw'] = grid['average_distance_nearest_building']
     grid['m2_raw'] = grid['m2_raw']#.fillna(grid['m2_raw'].mean())
     grid['m2_std'] = grid['m2_raw'].map_partitions(standardize_metric_2, meta=('m2', 'float64'))
     
     path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metrics_1_2.geoparquet'
    
     if 'geom' in grid.columns:
         grid = grid.drop(columns='geom')
    
     grid.to_parquet(path)

@delayed
def compute_m6_m7(city_name):
    """
    Computes:
    - M6: KL divergence (building orientation), weighted by:
         (a) block’s proportional overlap with each grid cell
         (b) number of buildings inside that block∩cell
      fallback: unweighted KL for cells with ≥2 buildings but no blocks
    - M7: Average block width
    """

    # 0) paths & load
    epsg = get_epsg(city_name).compute()
    grid = load_dataset(f'{GRIDS_PATH}/{city_name}/{city_name}_{grid_size}m_grid.geoparquet',
                        epsg=epsg)
    blocks    = load_dataset(f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet', epsg=epsg).persist()
    buildings = load_dataset(f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances_and_azimuths.geoparquet', epsg=epsg).persist()
    buildings['azimuth'] = buildings['azimuth']\
        .map_partitions(pd.to_numeric,
                        meta=('azimuth','float64'),
                        errors='coerce')


    # drop stray geometry column if present
    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])

    # 1) prepare blocks
    epsilon = 0.001
    blocks = blocks.assign(
        block_id=blocks.index,
        epsilon_buffer=blocks.geometry.buffer(-(1-epsilon)*blocks.max_radius),
        width_buffer  =blocks.geometry.buffer(-0.2*blocks.max_radius)
    )

    # 2) compute block→grid overlaps (with area_weight)
    #    compute_block_grid_weights returns a **pandas** GeoDataFrame after compute()
    bgo = compute_block_grid_weights(blocks, grid).compute()

    # 3) count **per (block, cell)** buildings inside each overlap
    #    (so blocks only contribute where they actually contain buildings)
    buildings_pdf = buildings.compute()[['geometry']]
    # join buildings → block_cell overlaps
    join = gpd.sjoin(buildings_pdf,
                     bgo[['block_id','grid_id','geometry']],
                     predicate='intersects')
    # count
    n_bc = (
        join
        .groupby(['block_id','grid_id'])
        .size()
        .rename('n_buildings_cell')
        .reset_index()
    )
    # merge back, fill zero
    bgo = (
        bgo
        .merge(n_bc, on=['block_id','grid_id'], how='left')
        .fillna({'n_buildings_cell': 0})
    )

    # 4) block-level KL & per-cell weighted m6
    kl_df   = compute_block_kl_metrics(
                  # still uses your buildings_blocks → block-level KL
                  dgpd.sjoin(buildings, blocks, predicate='intersects')
                     [['block_id','geometry','epsilon_buffer','width_buffer','azimuth']]
                     .set_index('block_id')
                     .repartition(npartitions=4)
              ).compute()
    # adjust aggregate to use n_buildings_cell
    df = (
        bgo
        .merge(kl_df, on='block_id', how='left')
        .dropna(subset=['standardized_kl'])
        .assign(weight = lambda d: d.area_weight * d.n_buildings_cell,
                weighted_kl = lambda d: d.standardized_kl * d.weight)
    )
    grid_m6 = (
        df
        .groupby('grid_id')
        .agg(total_weighted_kl=('weighted_kl','sum'),
             total_weight=('weight','sum'))
    )
    grid_m6['m6'] = grid_m6.total_weighted_kl / grid_m6.total_weight

    # 5) compute M7 as before
    bgo['weighted_max_radius'] = bgo.max_radius * bgo.area_weight
    grid_m7 = (
        bgo
        .groupby('grid_id')
        .agg(total_weighted_max_radius=('weighted_max_radius','sum'),
             total_weight=('area_weight', 'sum'))
    )
    grid_m7['m7'] = grid_m7.total_weighted_max_radius / grid_m7.total_weight

    # 6) unweighted KL fallback for cells with ≥2 buildings (no blocks)
    def kl_divergence(arr, bins=18):
        hist,_ = np.histogram(arr, bins=bins, range=(0,180))
        P = hist/hist.sum() if hist.sum()>0 else np.ones(bins)/bins
        Q = np.ones(bins)/bins
        return entropy(P, Q) / np.log(bins)

    b2g = dgpd.sjoin(buildings[['geometry','azimuth']],
                     grid[['geometry']],
                     predicate='intersects'
                    ).persist()
    m6_unw = (
        b2g.groupby('index_right')['azimuth']
           .apply(lambda s: kl_divergence(s.to_numpy()),
                  meta=('azimuth','float64'))
           .rename('m6_unweighted')
    )

    # 7) stitch everything back onto the full grid
    #    name all indexes “grid_id” so reset_index() yields a column you can join on
    grid        = grid       .rename_axis("grid_id")
    grid_m6     = grid_m6    .rename_axis("grid_id")
    m6_unw      = m6_unw     .rename_axis("grid_id")
    grid_m7     = grid_m7    .rename_axis("grid_id")

    grid = (
        grid
        .reset_index()                                # now has “grid_id” column
        .merge(grid_m6.reset_index(),   on="grid_id", how="left")
        .merge(m6_unw.reset_index(),    on="grid_id", how="left")
        .merge(grid_m7.reset_index()[['grid_id','m7']],
               on="grid_id", how="left")
        .set_index("grid_id")
    )

    # 8) build & standardize raw columns
    grid['m6_raw'] = grid['m6'].fillna(grid['m6_unweighted'])
    grid['m6_std'] = grid['m6_raw'].map_partitions(
                        standardize_metric_6, meta=('m6','float64')
                     )

    grid['m7_raw'] = grid['m7'].fillna(200)
    grid['m7_std'] = grid['m7_raw'].map_partitions(
                        standardize_metric_7, meta=('m7','float64')
                     )

    grid = grid.drop(columns=['m6','m7'])
    pdf  = grid.compute()

    # 9) write out
    out = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_grid_{YOUR_NAME}_metrics_6_7.geoparquet'
    pdf.to_parquet(out, engine='pyarrow', index=False)
    return out

@delayed
def metrics_roads_intersections(city_name):

    paths = {
    'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
    'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
    'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
    'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
    'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }

    # LOAD
    epsg = get_epsg(city_name).compute()
    grid = load_dataset(paths['grid'], epsg=epsg).persist()
    roads = load_dataset(paths['roads'], epsg=epsg).persist()
    intersections = load_dataset(paths['intersections'], epsg=epsg).compute()


    included_road_types = ['trunk','motorway','primary','secondary','tertiary','primary_link','secondary_link','tertiary_link','trunk_link','motorway_link','residential','unclassified','road','living_street']
    roads['highway'][roads['highway'].isin(included_road_types)]
    

    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])


    
    # Prep for metric 10
    intersections['osmid'] = intersections['osmid'].astype(int)
    intersection_angles = compute_intersection_angles(roads, intersections)
    street_count_mapping = intersections.set_index('osmid')['street_count'].to_dict()
    intersection_angle_mapping = compute_intersection_mapping(intersection_angles, street_count_mapping)
    intersection_angle_mapping = intersection_angle_mapping.compute()  

    intersections_with_angles_metric = intersections.merge(
        intersection_angle_mapping.rename("average_angle"), left_on="osmid", right_index=True, how="left"
    )

    joined_intersection_angles_grid = dgpd.sjoin(intersections_with_angles_metric, grid, predicate="within")
    average_angle_between_roads = joined_intersection_angles_grid.groupby('index_right')['average_angle'].mean()
    

    # Prep for metric 9
    
    roads_simple = roads[['geometry']]
    
    grid_small = (
        grid
        .reset_index()[["index","geometry"]]
        .rename(columns={"index":"index_right"})
    )
    
    # inside metrics_roads_intersections, before overlay:
    overlay_meta = gpd.GeoDataFrame(
        {
            "index_right": pd.Series(dtype="int64"),
            "geometry":    gpd.GeoSeries(dtype="geometry")
        },
        geometry="geometry"
    )

    roads_cells = roads_simple.map_partitions(
        overlay_partition,
        grid_small,
        meta=overlay_meta
    ).persist()

    # roads_cells now has:
    #  - geometry   = clipped road piece
    #  - index_right = the cell it belongs to

    # 3) compute wt & length in one pass per partition
    out = roads_cells.map_partitions(
        partition_tortuosity_clipped,   
        meta=pd.DataFrame({
            "index_right": pd.Series(dtype="int64"),
            "wt":           pd.Series(dtype="float64"),
            "length":      pd.Series(dtype="float64")
        })
    ).persist()

    # 4) aggregate back into grid
    agg = out.groupby("index_right").agg(
        total_len=("length","sum"),
        sum_wt   =("wt",    "sum")
    )
    grid["m8_raw"] = grid.index.map(agg["sum_wt"]/agg["total_len"]).astype(float)
    grid["m8_std"] = grid["m8_raw"].map_partitions(
        standardize_metric_8, meta=("m8","float64")
    )
    
    grid['m9_raw'] = grid.index.map(average_angle_between_roads).astype(float)#.fillna(np.mean(average_angle_between_roads)).astype(float)
    grid['m9_std'] = grid['m9_raw'].map_partitions(standardize_metric_9, meta=('m9', 'float64'))
    
    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_metrics_8_9_{YOUR_NAME}.geoparquet'

    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')

    grid.to_parquet(path)

    return path


In [10]:
import time
from dask import compute

start_time = time.time()  # Start the timer

#cities = ["Abidjan", "Accra", "Bamako", "Belo_Horizonte","Bogota", 'Nairobi',"Campinas", "Cape_Town", "Luanda", "Medellin"]
cities = ['Medellin']
#"Lagos"
cities = [city.replace(' ', '_') for city in cities]

tasks = []
for city in cities:
    tasks.append(building_and_intersection_metrics(city))
    tasks.append(building_distance_metrics(city))
    tasks.append(compute_m6_m7(city))
    tasks.append(metrics_roads_intersections(city))

results = compute(*tasks)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

Tasks completed in 33.83 seconds.


In [11]:
import dask_geopandas as dgpd
import pandas as pd
import numpy as np
from shapely import wkb

def consolidate_irregularity_index(city_name):
    # 1) Paths (same as before) …
    base = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m'
    p0 = f'{base}_metrics_3_4_5_10_11_12_grid_{YOUR_NAME}.geoparquet'
    p1 = f'{base}_grid_{YOUR_NAME}_metrics_1_2.geoparquet'
    p2 = f'{base}_grid_{YOUR_NAME}_metrics_6_7.geoparquet' 
    p3 = f'{base}_grid_metrics_8_9_{YOUR_NAME}.geoparquet'

    # 2) Lazy-read  
    g0 = dgpd.read_parquet(p0)
    g1 = dgpd.read_parquet(p1)[['m1_std','m2_std','m1_raw','m2_raw','has_buildings']].drop(columns='geometry', errors='ignore')
    g2 = dgpd.read_parquet(p2)[['m6_std','m7_std','m6_raw','m7_raw']].drop(columns='geometry', errors='ignore')
    g3 = dgpd.read_parquet(p3)[['m8_std','m9_std','m8_raw','m9_raw']].drop(columns='geometry', errors='ignore')

    # 3) Merge (keeps geometry)
    df = (g0
          .merge(g1, left_index=True, right_index=True, how='left')
          .merge(g2, left_index=True, right_index=True, how='left')
          .merge(g3, left_index=True, right_index=True, how='left'))

    # 4) Mask
    std_cols = [f'm{i}_std' for i in range(1,13)]
    df['has_features'] = df['has_buildings'] | df['has_roads'] | df['has_intersections']
    df[std_cols] = df[std_cols].where(df['has_features'], np.nan)

    # 5) Persist & repartition
    df = df.persist().repartition(npartitions=8)

    all_metrics_columns_raw = ['m'+str(x)+'_raw' for x in range(1,13)]
    all_metrics_columns_final = ['m'+str(x)+'_final' for x in range(1,13)]
    all_metrics_columns_std = ['m'+str(x)+'_std' for x in range(1,13)]
    all_metrics_columns_zc = ['m'+str(x)+'_zc' for x in range(1,13)]
    
    # 6) Compute global scalars on standardized metrics
    means = df[all_metrics_columns_std].mean().compute()
    stds  = df[all_metrics_columns_std].std().compute()
    mins  = df[all_metrics_columns_std].min().compute()
    maxs  = df[all_metrics_columns_std].max().compute()

    # Convert those into z-mins / z-maxs and re-index to your zc_cols
    zmin = (mins - means) / stds
    zmax = (maxs - means) / stds
    zmin.index = all_metrics_columns_zc
    zmax.index = all_metrics_columns_zc

    # 7) One-pass normalize + index
    def normalize_partition(pdf, means, stds, zmin, zmax):
        # zero-center -> z DataFrame
        z = (pdf[all_metrics_columns_std] - means) / stds
        z.columns = all_metrics_columns_zc
        pdf[all_metrics_columns_zc] = z

        # min–max on z -> f DataFrame
        f = (z - zmin) / (zmax - zmin)
        f.columns = all_metrics_columns_final
        pdf[all_metrics_columns_final] = f

        groupA = [f"m{i}_final" for i in range(1,10)]        # metrics 1–9
        groupB = groupA + ["m12_final"]                     # metrics 1–9 plus 12

        # … after pdf[all_metrics_columns_final] = f …
        # compute two regularity indices and their NA‐counts
        pdf["regularity_index_A"] = f[groupA].mean(axis=1)
        pdf["na_count_A"]          = f[groupA].isna().sum(axis=1)

        pdf["regularity_index_B"] = f[groupB].mean(axis=1)
        pdf["na_count_B"]          = f[groupB].isna().sum(axis=1)
        return pdf


    # 8) Build meta and map_partitions
    meta = df._meta.copy()
    for c in (
        all_metrics_columns_zc
      + all_metrics_columns_final
      + ['regularity_index_A','na_count_A','regularity_index_B','na_count_B']
    ):
        meta[c] = pd.Series(dtype='float64')


    df = df.map_partitions(
        normalize_partition,
        means, stds, zmin, zmax,
        meta=meta
    )

    import geopandas as gpd
    
    out = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_all_metrics_combined_{YOUR_NAME}.geoparquet'
    # … after your df = df.map_partitions(...), instead of WKB+pd.to_parquet:
    pdf = df.compute()

    # rehydrate into a GeoDataFrame, borrow the original CRS:
    gdf = gpd.GeoDataFrame(pdf, geometry='geometry', crs=g0.crs)

    # write **one** parquet file, with embedded CRS and GeoArrow extension types:
    gdf.to_parquet(out, engine='pyarrow', index=False)

    '''
    # 9) Compute into pandas, then convert geometry to WKB
    pdf = df.compute()
    pdf['geometry'] = pdf.geometry.apply(lambda geom: wkb.dumps(geom, hex=False))

    # 10) Write one parquet file via pandas (supports single_file)
    
    pdf.to_parquet(out, engine='pyarrow', index=False)#single_file=True, 
    #df.to_parquet(out, engine='pyarrow', index=False)#
    '''
    return out

# Dispatch in parallel:
#cities = ["Abidjan","Accra","Bamako","Belo_Horizonte","Bogota","Nairobi","Campinas","Cape_Town","Luanda","Medellin"]
cities = ['Medellin']
futures = client.map(consolidate_irregularity_index, cities)
results = client.gather(futures)
print("Written outputs:", results)


Written outputs: ['s3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Medellin/Medellin_200m_all_metrics_combined_sara.geoparquet']


In [ ]:
import geopandas as gpd
import pandas as pd

cities =["Abidjan","Accra","Bamako","Belo_Horizonte","Bogota", "Nairobi","Campinas","Cape_Town","Luanda","Medellin"]
paths = [f'{OUTPUT_PATH_RASTER}/{city}/{city}_{grid_size}m_all_metrics_combined_{YOUR_NAME}.geoparquet' 
         for city in cities]

# Read into GeoPandas and concat
gdfs = [gpd.read_parquet(path) for path in paths]
combined = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Write a single GeoPackage file
combined.to_file('all_cities_combined.gpkg', driver='GPKG')
print("Combined GeoPackage saved as all_cities_combined.gpkg")


In [ ]:
grid_combined.to_parquet('../Medellin_consolidated_index_results.parquet')#regularity_index.describe() #[['m1_raw','m2_raw','regularity_index']]

In [26]:
grid_combined.to_file('../Medellin_consolidated_index_results.shp')

/var/folders/mt/3n9j2kc92kv4psztx687vtd80000gn/T/ipykernel_71266/401492889.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_combined.to_file('../Medellin_consolidated_index_results.shp')
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'cell_area_km2' to 'cell_area_'
  ogr_write(
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'n_buildings' to 'n_building'
  ogr_write(
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'road_length' to 'road_lengt'
  ogr_write(
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'n_intersections' to 'n_intersec'
  ogr_wr

In [21]:
mde_grid = gpd.read_parquet('../Medellin_consolidated_index_results.parquet')

In [19]:
grid_combined[['m1_raw','m1_std','m1_zero-centered','m1_final']].describe()

,m1_raw,m1_std,m1_zero-centered,m1_final
count,10207.000000,10207.000000,1.332400e+04,8563.000000
mean,0.498136,0.498136,-5.119491e-17,0.357416
std,0.331203,0.331203,1.000000e+00,0.390134
min,0.000000,0.000000,-7.411228e+00,0.000000
25%,0.200000,0.200000,-1.824746e-01,0.000000
50%,0.540541,0.540541,3.998596e-01,0.165408
75%,0.782609,0.782609,6.608733e-01,0.765262
max,1.000000,1.000000,6.809505e-01,1.000000


In [20]:
grid_combined[['m2_raw','m2_std','m2_zero-centered','m2_final']].describe()

,m2_raw,m2_std,m2_zero-centered,m2_final
count,10207.000000,10207.000000,1.332400e+04,9090.000000
mean,52.151067,0.649161,4.266243e-17,0.094862
std,89.740864,0.299055,1.000000e+00,0.143445
min,0.000000,0.000000,-6.866936e-01,0.000000
25%,13.234007,0.544541,-6.832878e-01,0.005162
50%,22.771024,0.772290,-4.674279e-01,0.036609
75%,45.545923,0.867660,4.301169e-01,0.119164
max,1223.135659,1.000000,1.193408e+01,1.000000


In [22]:
grid_combined[['m8_raw','m8_std','m8_zero-centered','m8_final']].describe()

,m8_raw,m8_std,m8_zero-centered,m8_final
count,9090.000000,9090.000000,1.332400e+04,10207.000000
mean,0.946877,0.053123,5.119491e-17,0.649161
std,0.080329,0.080329,1.000000e+00,0.299055
min,0.440000,0.000000,-7.310172e-01,0.000000
25%,0.933268,0.002891,-7.310172e-01,0.544541
50%,0.979499,0.020501,-4.778693e-01,0.772290
75%,0.997109,0.066732,5.349885e-01,0.867660
max,1.000000,0.560000,2.055812e+00,1.000000


In [27]:
mde_grid[['regularity_index']].describe()

,regularity_index
count,10844.000000
mean,0.185482
std,0.100525
min,0.083352
25%,0.118102
50%,0.150246
75%,0.212495
max,0.698691
